<a href="https://colab.research.google.com/github/buzz-lightsnack-2007/Desktop-CoLab/blob/main/DesktopCoLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desktop@CoLab

**A Linux virtual machine, all in Google's servers.**

<small>_Uses [remocolab](https://urlzs.com/jBN9S) by [demotomohiro](https://urlzs.com/ds5GF)._</small>

---


_**Important**: To start a new instance using the same account, please re-set the runtime. Forgetting to do so will cause an error._ 

---

In [ ]:
# DESKTOP@COLAB
# Have a Linux virtual machine hosted by Google. 

#@title ## Configuration
class config: 
  connection = "SSH only" #@param ["SSH only", "VNC"]

  class software: 
  #@markdown ##Software  
    upgrade_all_binaries = 0 #@param {type:"slider", min:0, max:1, step:1}

  class storage: 
    #@markdown ##Storage
    #@markdown ### Google Drive Storage Integration
    use = 0 #@param {type:"slider", min:0, max:1, step:1}
    drive_root = 'My Drive' #@param ['My Drive', 'Shared drives'] 
    folder = "" #@param {type:"string"}
    drive_folder = '/'.join((drive_root, folder)) # Actual Google Drive folder
    mount_point = '' #@param {type:"string"}

# actual runtime
## Import modules. 
import os

## Set formatting.
class formatting:
  clear = '\033[0m'
  class color: 
    purple = '\033[95m'
    cyan = '\033[96m'
    cyan_dark = '\033[36m'
    blue = '\033[94m'
    green = '\033[92m'
    yellow = '\033[93m'
    red = '\033[91m'
  class font:
    bold = '\033[1m'
    underline = '\033[4m'

## Define screen functions. 
def clear(): 
  from IPython.display import clear_output; return clear_output()

class info: # screen messages
  def status(message):
    print(formatting.font.bold + 'Status: \t' + formatting.clear + message)
  def err(message):
    print(formatting.font.bold + formatting.color.red + 'Error: \t' + formatting.clear + formatting.clear + message)
  def warning(message):
    print(formatting.font.bold + formatting.color.yellow + 'Warning: \t' + formatting.clear + formatting.clear + message)
  def success(message):
    print(formatting.font.bold + formatting.color.green + 'Success: \t' + formatting.clear + formatting.clear + message)


#try:
def mount_GDrive():
  ## Configure storage. 
  info.status('Google Drive integration has been enabled! To properly integrate to your Google Drive, please accept the Google Account permission prompts.')
  
  try:
    from google.colab import drive
    info.status('Connecting to Google Drive…')
    drive.mount('/content/drive')
  
  except:
    config.storage.use = 0
    info.error('Google Drive could not be connected.')
    info.status('Google Drive integration has been disabled for this instance. ')

  else: 
    info.success('Google Drive has been connected.')
    clear()

# Upgrade necessary components. 
def update_components():
  os.system('add-apt-repository ppa:apt-fast/stable -y && apt update && apt install apt-fast -y && apt-fast clean && apt-fast autoremove -y')

  if config.software.upgrade_all_binaries:
    info.status('Upgrading all binaries.')
    try:
      if os.system('apt-fast update && apt-fast upgrade -y && apt-fast clean && apt-fast autoremove -y') > 0: raise
    except:
      info.err('An exception has been raised when trying to upgrade one or more of the binaries.')
    else: 
      info.success('All binaries have been upgraded!')

def install_remoColab():
  # Install RemoColab. 
  info.status('Installing RemoColab…')
  try:
    os.system('pip install git+https://github.com/demotomohiro/remocolab.git')
    import remocolab
  except:
    info.err('Could not install RemoColab.')
    os.kill(os.getpid(), 1)
  else:  
    info.success('Successfully installed RemoColab.')

def establish_connection():
  import remocolab

  def install_wm():
    # install xfce. 
    info.status('Installing XFCE.')
    try:
      if os.system('apt-fast install xfce4 xfce4-terminal -y') > 0: 
        if os.system('apt install xfce4 xfce4-terminal -y') > 0: raise
    except:
      info.err('Could not install the XFCE desktop.')
      config.connection = 'SSH only'; info.status('VNC connection will be disabled.')
    else:
      info.success('Successfully installed the XFCE desktop.')

  if config.connection == 'VNC': install_wm()

  info.status('Setting up ' + config.connection + ' connection. ')
  if config.connection == 'VNC':
    if config.storage.use == 1:
      if config.storage.mount_point != None:
        remocolab.setupVNC(tunnel = "argotunnel", mount_gdrive_from = config.storage.drive_folder, mount_gdrive_to = 'Google Drive')
      else:  
        remocolab.setupVNC(tunnel = "argotunnel", mount_gdrive_from = config.storage.drive_folder, mount_gdrive_to = 'mount_point')
    else:
      remocolab.setupVNC(tunnel = "argotunnel")
    info.success('The VNC connection is available at ' + formatting.font.bold + 'vnc://localhost:5901' + formatting.clear + ' unless stated otherwise.')
  else:
    if config.storage.use == 1:
      try:
        if config.storage.mount_point != None:
          remocolab.setupSSHD(tunnel = "argotunnel", mount_gdrive_from = config.storage.drive_folder, mount_gdrive_to = 'Google Drive')
        else:  
          remocolab.setupSSHD(tunnel = "argotunnel", mount_gdrive_from = config.storage.drive_folder, mount_gdrive_to = 'mount_point')
      except:
        remocolab.setupSSHD(tunnel = "argotunnel")
    else:
      remocolab.setupSSHD(tunnel = "argotunnel")

def main(): 
  if config.storage.use: mount_GDrive()
  update_components()
  install_remoColab()
  establish_connection()

main()